# EOF Analysis of Pacific SST Anomalies

**Publication-quality visualization for Nature/Science journals**

This notebook demonstrates:
- Generation of synthetic SST data
- EOF (Empirical Orthogonal Function) analysis
- Professional visualization with matplotlib and cartopy

Author: Climate Data Analysis

Date: December 2025

In [ ]:
# Install required packages (for Binder)
import sys
!{sys.executable} -m pip install -q numpy matplotlib cartopy cmocean

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
try:
    import cartopy.crs as ccrs
    import cartopy.feature as cfeature
    HAS_CARTOPY = True
except ImportError:
    HAS_CARTOPY = False
    print("Cartopy not available, using basic plots")

try:
    import cmocean
    HAS_CMOCEAN = True
except ImportError:
    HAS_CMOCEAN = False
    print("cmocean not available, using standard colormaps")

# Set publication-quality defaults
plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.sans-serif': ['Arial', 'Helvetica', 'DejaVu Sans'],
    'font.size': 8,
    'axes.linewidth': 0.5,
    'xtick.major.width': 0.5,
    'ytick.major.width': 0.5,
    'figure.dpi': 100,
    'savefig.dpi': 300,
    'savefig.bbox': 'tight'
})

print("✓ Libraries imported successfully")

## 1. Generate Synthetic SST Data

We simulate Pacific SST anomalies with three main modes:
1. **ENSO-like pattern** (35% variance) - equatorial Pacific oscillation
2. **PDO-like pattern** (19% variance) - North Pacific decadal variability
3. **High-frequency mode** (9% variance) - interannual variability

In [ ]:
def generate_synthetic_eof_data():
    """
    Generate synthetic EOF analysis data for demonstration.
    
    Returns:
    --------
    dict with keys: eofs, pcs, variance_explained, lats, lons, time
    """
    np.random.seed(42)  # For reproducibility
    
    # Grid setup
    nlat = 40
    nlon = 60
    ntime = 300  # 25 years of monthly data
    
    lats = np.linspace(-60, 60, nlat)
    lons = np.linspace(120, 290, nlon)
    time = np.linspace(1950, 2025, ntime)
    
    LON, LAT = np.meshgrid(lons, lats)
    
    # EOF 1: ENSO-like pattern (tropical Pacific)
    centered_lon = LON - 200
    eof1 = np.exp(-(centered_lon**2/3000 + LAT**2/1000)) * np.cos(centered_lon * 0.02) * 1.5
    
    # EOF 2: PDO-like pattern (North Pacific)
    eof2 = np.sin((LON - 180) * 0.03) * np.cos(LAT * 0.04)
    eof2 = np.where(LAT > 0, eof2, eof2 * 0.7)
    
    # EOF 3: Higher frequency spatial pattern
    eof3 = np.sin(LON * 0.05) * np.sin(LAT * 0.08) * 0.8
    
    eofs = [eof1, eof2, eof3]
    
    # Principal Components (time series)
    t_index = np.arange(ntime)
    
    # PC1: ENSO-like (3-7 year oscillation)
    pc1 = (np.sin(t_index * 0.15) * 2 + 
           np.sin(t_index * 0.03) * 3 + 
           np.random.randn(ntime) * 0.5)
    
    # PC2: Decadal variability
    pc2 = (np.sin(t_index * 0.02) * 2.5 + 
           np.cos(t_index * 0.08) * 1.5 + 
           np.random.randn(ntime) * 0.4)
    
    # PC3: Higher frequency
    pc3 = (np.sin(t_index * 0.3) * 1.5 + 
           np.random.randn(ntime) * 0.8)
    
    pcs = [pc1, pc2, pc3]
    
    # Variance explained
    variance_explained = np.array([35.2, 18.7, 9.3])
    
    return {
        'eofs': eofs,
        'pcs': pcs,
        'variance_explained': variance_explained,
        'lats': lats,
        'lons': lons,
        'time': time
    }

# Generate data
data = generate_synthetic_eof_data()

print(f"✓ Generated synthetic data:")
print(f"  - Spatial grid: {len(data['lats'])}°lat × {len(data['lons'])}°lon")
print(f"  - Time series: {len(data['time'])} months ({data['time'][0]:.0f}-{data['time'][-1]:.0f})")
print(f"  - Total variance explained by first 3 modes: {data['variance_explained'].sum():.1f}%")

## 2. Visualization Functions

Define publication-quality plotting functions

In [ ]:
def plot_eof_spatial(eof, lons, lats, mode_num, variance, ax=None):
    """
    Plot EOF spatial pattern with or without cartopy.
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 4))
    
    # Color levels centered on zero
    vmax = np.abs(eof).max()
    levels = np.linspace(-vmax, vmax, 21)
    
    # Choose colormap
    if HAS_CMOCEAN:
        cmap = cmocean.cm.balance
    else:
        cmap = 'RdBu_r'
    
    if HAS_CARTOPY and isinstance(ax, cartopy.mpl.geoaxes.GeoAxes):
        # Cartopy plot
        cf = ax.contourf(lons, lats, eof, levels=levels, cmap=cmap,
                        transform=ccrs.PlateCarree(), extend='both')
        ax.coastlines(linewidth=0.5)
        ax.add_feature(cfeature.LAND, facecolor='lightgray', zorder=0)
        ax.gridlines(draw_labels=True, linewidth=0.3, alpha=0.5, linestyle='--')
    else:
        # Basic matplotlib plot
        cf = ax.contourf(lons, lats, eof, levels=levels, cmap=cmap, extend='both')
        ax.set_xlabel('Longitude (°E)', fontsize=8)
        ax.set_ylabel('Latitude (°N)', fontsize=8)
        ax.grid(True, alpha=0.3, linewidth=0.3)
    
    ax.set_title(f'EOF {mode_num} ({variance:.1f}% variance)', 
                fontsize=9, fontweight='bold', pad=8)
    
    return cf

def plot_pc_timeseries(pc, time, mode_num, ax=None):
    """
    Plot normalized PC time series.
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(6, 2.5))
    
    # Normalize
    pc_norm = (pc - pc.mean()) / pc.std()
    
    # Fill positive/negative
    ax.fill_between(time, 0, pc_norm, where=(pc_norm >= 0),
                    color='#d73027', alpha=0.7, label='Positive phase')
    ax.fill_between(time, 0, pc_norm, where=(pc_norm < 0),
                    color='#4575b4', alpha=0.7, label='Negative phase')
    
    # Line on top
    ax.plot(time, pc_norm, 'k-', linewidth=0.5, alpha=0.8)
    
    # Zero line
    ax.axhline(0, color='black', linewidth=0.5, alpha=0.3)
    
    ax.set_xlabel('Year', fontsize=8)
    ax.set_ylabel(f'PC{mode_num} (normalized)', fontsize=8)
    ax.grid(True, alpha=0.2, linewidth=0.3)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.legend(fontsize=6, frameon=False, loc='upper right')
    
    return ax

print("✓ Plotting functions defined")

## 3. Create Multi-Panel Summary Figure

This produces a Nature/Science-style figure with:
- Left column: EOF spatial patterns
- Right column: PC time series
- Panel labels (a, b, c, d, e, f)

In [ ]:
# Create figure
fig = plt.figure(figsize=(10, 11))
gs = gridspec.GridSpec(3, 2, figure=fig, width_ratios=[1.5, 1], 
                       hspace=0.35, wspace=0.35)

for i in range(3):
    # Spatial pattern (left)
    if HAS_CARTOPY:
        ax_map = fig.add_subplot(gs[i, 0], projection=ccrs.PlateCarree(central_longitude=180))
        ax_map.set_extent([120, 290, -60, 60], crs=ccrs.PlateCarree())
    else:
        ax_map = fig.add_subplot(gs[i, 0])
    
    cf = plot_eof_spatial(data['eofs'][i], data['lons'], data['lats'], 
                         i+1, data['variance_explained'][i], ax=ax_map)
    
    # Colorbar
    cbar = plt.colorbar(cf, ax=ax_map, orientation='horizontal', 
                       pad=0.08, aspect=25, shrink=0.85)
    cbar.set_label('SST anomaly (°C)', fontsize=7)
    cbar.ax.tick_params(labelsize=6, width=0.3)
    
    # Panel label
    letter = chr(97 + i*2)  # a, c, e
    ax_map.text(-0.08, 1.05, f'({letter})', transform=ax_map.transAxes,
               fontsize=11, fontweight='bold', va='top')
    
    # Time series (right)
    ax_ts = fig.add_subplot(gs[i, 1])
    plot_pc_timeseries(data['pcs'][i], data['time'], i+1, ax=ax_ts)
    
    # Panel label
    letter = chr(98 + i*2)  # b, d, f
    ax_ts.text(-0.15, 1.05, f'({letter})', transform=ax_ts.transAxes,
              fontsize=11, fontweight='bold', va='top')

plt.suptitle('EOF Analysis of Pacific SST Anomalies', 
            fontsize=12, fontweight='bold', y=0.995)

plt.savefig('eof_summary_figure.pdf', dpi=300, bbox_inches='tight')
plt.savefig('eof_summary_figure.png', dpi=300, bbox_inches='tight')
print("✓ Figure saved as 'eof_summary_figure.pdf' and 'eof_summary_figure.png'")

plt.show()

## 4. Scree Plot - Variance Explained

Shows the contribution of each EOF mode to total variance

In [ ]:
fig, ax = plt.subplots(figsize=(5, 3.5))

modes = np.arange(1, 4)
variance = data['variance_explained']

# Bar plot
bars = ax.bar(modes, variance, color='steelblue', alpha=0.7, 
              edgecolor='black', linewidth=0.5, width=0.6)

# Cumulative line
ax2 = ax.twinx()
cumsum = np.cumsum(variance)
ax2.plot(modes, cumsum, 'ro-', linewidth=2, markersize=6, label='Cumulative')
ax2.set_ylabel('Cumulative variance (%)', fontsize=9, color='red')
ax2.tick_params(labelsize=8, colors='red', width=0.5)
ax2.spines['top'].set_visible(False)

# Styling
ax.set_xlabel('EOF mode', fontsize=9)
ax.set_ylabel('Variance explained (%)', fontsize=9)
ax.set_xticks(modes)
ax.tick_params(labelsize=8, width=0.5)
ax.spines['top'].set_visible(False)
ax.grid(True, alpha=0.3, linewidth=0.3, axis='y')

ax.set_title('Variance Explained by EOF Modes', fontsize=10, fontweight='bold', pad=10)

# Add percentage labels on bars
for i, (bar, v) in enumerate(zip(bars, variance)):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.5,
           f'{v:.1f}%', ha='center', va='bottom', fontsize=7)

plt.tight_layout()
plt.savefig('scree_plot.pdf', dpi=300, bbox_inches='tight')
plt.savefig('scree_plot.png', dpi=300, bbox_inches='tight')
print("✓ Scree plot saved as 'scree_plot.pdf' and 'scree_plot.png'")

plt.show()

print(f"\nTotal variance explained by first 3 modes: {cumsum[-1]:.1f}%")

## 5. Individual EOF Modes (High Resolution)

Create individual high-resolution plots for each mode

In [ ]:
for i in range(3):
    fig = plt.figure(figsize=(8, 5))
    
    if HAS_CARTOPY:
        ax = fig.add_subplot(111, projection=ccrs.PlateCarree(central_longitude=180))
        ax.set_extent([120, 290, -60, 60], crs=ccrs.PlateCarree())
    else:
        ax = fig.add_subplot(111)
    
    cf = plot_eof_spatial(data['eofs'][i], data['lons'], data['lats'],
                         i+1, data['variance_explained'][i], ax=ax)
    
    cbar = plt.colorbar(cf, ax=ax, orientation='horizontal',
                       pad=0.08, aspect=30, shrink=0.8)
    cbar.set_label('SST anomaly (°C)', fontsize=9)
    cbar.ax.tick_params(labelsize=8)
    
    plt.savefig(f'eof{i+1}_spatial.pdf', dpi=300, bbox_inches='tight')
    plt.savefig(f'eof{i+1}_spatial.png', dpi=300, bbox_inches='tight')
    print(f"✓ EOF {i+1} spatial pattern saved")
    
    plt.show()
    plt.close()

## 6. Figure Caption for Publication

**Figure 1. Leading modes of SST variability from EOF analysis.**

(a,c,e) Spatial patterns of EOF modes 1-3, showing the correlation between SST anomalies and the principal component. Colors indicate anomaly magnitude in °C, with red (blue) indicating warming (cooling) associated with positive phase of the mode. Contour interval determined by data range. 

(b,d,f) Corresponding normalized principal component time series. Red (blue) shading indicates positive (negative) phase. Black line shows the time evolution of each mode. 

Variance explained by each mode is indicated in panel titles. The first three modes explain 63.2% of total variance in Pacific SST anomalies.

**Data:** Simulated Pacific SST dataset, 1950-2024, monthly anomalies relative to long-term mean. Grid resolution: 3°×3°.

---

## Summary Statistics

In [ ]:
print("="*60)
print("EOF ANALYSIS SUMMARY")
print("="*60)
print(f"\nData characteristics:")
print(f"  Latitude range: {data['lats'][0]:.1f}°N to {data['lats'][-1]:.1f}°N")
print(f"  Longitude range: {data['lons'][0]:.1f}°E to {data['lons'][-1]:.1f}°E")
print(f"  Time period: {data['time'][0]:.0f} to {data['time'][-1]:.0f}")
print(f"  Number of time steps: {len(data['time'])}")

print(f"\nVariance explained by EOF modes:")
for i, v in enumerate(data['variance_explained']):
    print(f"  EOF {i+1}: {v:.1f}%")

cumsum = np.cumsum(data['variance_explained'])
print(f"\nCumulative variance:")
for i, v in enumerate(cumsum):
    print(f"  First {i+1} mode(s): {v:.1f}%")

print(f"\nPrincipal component statistics:")
for i, pc in enumerate(data['pcs']):
    pc_norm = (pc - pc.mean()) / pc.std()
    print(f"  PC{i+1}: mean={pc.mean():.3f}, std={pc.std():.3f}, "
          f"min={pc_norm.min():.2f}, max={pc_norm.max():.2f}")

print("\n" + "="*60)
print("Files saved:")
print("  - eof_summary_figure.pdf/png (multi-panel)")
print("  - scree_plot.pdf/png")
print("  - eof1_spatial.pdf/png")
print("  - eof2_spatial.pdf/png")
print("  - eof3_spatial.pdf/png")
print("="*60)